In [4]:
!pip install factor_analyzer

     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 346.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for factor_analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42715 sha256=adeab276c80db292668ed1bef88d825a40e536a2629298c82bf827abd46546c4
  Stored in directory: c:\users\divig\appdata\local\pip\cache\wheels\fa\f7\53\a55a8a56668a6fe0199e0e02b6e0ae3007ec35cdf6e4c25df7
Successfully built factor_analyzer


In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy import stats
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.multivariate.manova import MANOVA
import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

In [10]:
# Load the data
df = pd.read_csv('NDAP_REPORT_7057_UPDATED.csv')

# Display basic info
print("Dataset Preview:")
print(df.head())
print("\nDataset Shape:", df.shape)
print("\nDescriptive Statistics:")
print(df.describe())
print("\nMissing Values:")
print(df.isnull().sum())

# Simplify Year
df['Year_Simple'] = df['Year'].apply(lambda x: '2017' if '2017' in x else '2021')


Dataset Preview:
   ROWID Country              State   District  \
0      1   India  Jammu And Kashmir   Anantnag   
1      2   India  Jammu And Kashmir   Anantnag   
2      3   India  Jammu And Kashmir     Budgam   
3      4   India  Jammu And Kashmir     Budgam   
4      5   India  Jammu And Kashmir  Baramulla   

                              Year  Class  Schools_Surveyed  \
0  Calendar Year (Jan - Dec), 2017      8             204.0   
1  Calendar Year (Jan - Dec), 2021      8             320.0   
2  Calendar Year (Jan - Dec), 2017      8             204.0   
3  Calendar Year (Jan - Dec), 2021      8             292.0   
4  Calendar Year (Jan - Dec), 2017      8             172.0   

   Students_Surveyed  AVG_L813  AVG_M601  AVG_SCI703  AVG_SST605  AVG_M801  \
0             2076.0     35.00     18.97       29.40       22.74     16.92   
1             4964.0     64.87     62.03       51.32       50.40     45.54   
2             2604.0     48.63     40.00       42.65       45.01     

In [12]:
performance_metrics = ['AVG_L813', 'AVG_M601', 'AVG_SCI703', 'AVG_SST605', 
                       'AVG_M801', 'AVG_SCI801', 'AVG_SST704']

corr = df[performance_metrics].corr()
print("\nCorrelation Matrix:")
print(corr)

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Correlation Matrix of Performance Metrics')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()



Correlation Matrix:
            AVG_L813  AVG_M601  AVG_SCI703  AVG_SST605  AVG_M801  AVG_SCI801  \
AVG_L813    1.000000  0.419171    0.737325    0.639266  0.474365    0.752299   
AVG_M601    0.419171  1.000000    0.515621    0.368070  0.655382    0.454589   
AVG_SCI703  0.737325  0.515621    1.000000    0.737461  0.551498    0.761532   
AVG_SST605  0.639266  0.368070    0.737461    1.000000  0.456965    0.661858   
AVG_M801    0.474365  0.655382    0.551498    0.456965  1.000000    0.501122   
AVG_SCI801  0.752299  0.454589    0.761532    0.661858  0.501122    1.000000   
AVG_SST704  0.564765  0.525135    0.528975    0.481501  0.457509    0.423429   

            AVG_SST704  
AVG_L813      0.564765  
AVG_M601      0.525135  
AVG_SCI703    0.528975  
AVG_SST605    0.481501  
AVG_M801      0.457509  
AVG_SCI801    0.423429  
AVG_SST704    1.000000  


In [14]:
# Boxplots
plt.figure(figsize=(14, 8))
df[performance_metrics].boxplot()
plt.title('Distribution of Academic Performance Metrics')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('performance_boxplots.png')
plt.close()

# Average by year
year_avg = df.groupby('Year_Simple')[performance_metrics].mean().T
year_avg.plot(kind='bar', figsize=(14, 8))
plt.title('Average Academic Performance by Year')
plt.ylabel('Average Score')
plt.xlabel('Subject')
plt.legend(title='Year')
plt.tight_layout()
plt.savefig('performance_by_year.png')
plt.close()

In [16]:
# Top 5 states
top_states = df['State'].value_counts().head(5).index.tolist()
print(f"\nTop 5 States by Number of Districts: {top_states}")

# State-wise plot
state_avg = df[df['State'].isin(top_states)].groupby('State')[performance_metrics].mean()
state_avg.plot(kind='bar', figsize=(14, 8))
plt.title('Average Academic Performance by State')
plt.ylabel('Average Score')
plt.xlabel('State')
plt.legend(title='Subject', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('performance_by_state.png')
plt.close()


Top 5 States by Number of Districts: ['Uttar Pradesh', 'Madhya Pradesh', 'Bihar', 'Maharashtra', 'Telangana']


In [18]:
from sklearn.decomposition import PCA

# Standardization
scaler = StandardScaler()
scaled_performance = scaler.fit_transform(df[performance_metrics])

# PCA Fit
pca = PCA()
pca_result = pca.fit_transform(scaled_performance)

# Variance Analysis
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Individual explained variance')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative explained variance')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% Threshold')
plt.legend()
plt.tight_layout()
plt.savefig('pca_explained_variance.png')
plt.close()

print("\nPCA Explained Variance Ratio:", explained_variance)
print("\nCumulative Explained Variance:", cumulative_variance)


PCA Explained Variance Ratio: [0.62307694 0.13112674 0.08348098 0.05586863 0.04703564 0.02994494
 0.02946613]

Cumulative Explained Variance: [0.62307694 0.75420368 0.83768466 0.89355329 0.94058893 0.97053387
 1.        ]


In [20]:
# Loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_df = pd.DataFrame(loadings, index=performance_metrics, columns=[f'PC{i+1}' for i in range(loadings.shape[1])])
print("\nPCA Component Loadings:")
print(loading_df)

# Loading Plot
plt.figure(figsize=(10, 8))
for i, metric in enumerate(performance_metrics):
    plt.arrow(0, 0, loading_df.loc[metric, 'PC1'], loading_df.loc[metric, 'PC2'], head_width=0.05, head_length=0.05, fc='blue', ec='blue')
    plt.text(loading_df.loc[metric, 'PC1'] * 1.15, loading_df.loc[metric, 'PC2'] * 1.15, metric, color='black', ha='center', va='center')
plt.grid(True)
plt.axhline(y=0, color='k', linestyle='--')
plt.axvline(x=0, color='k', linestyle='--')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.xlabel(f'PC1 ({explained_variance[0]:.2%})')
plt.ylabel(f'PC2 ({explained_variance[1]:.2%})')
plt.title('PCA Loading Plot')
plt.tight_layout()
plt.savefig('pca_loading_plot.png')
plt.close()

# Scatter by year
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=df['Year_Simple'].map({'2017': 0, '2021': 1}), cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Year (0: 2017, 1: 2021)')
plt.xlabel(f'PC1 ({explained_variance[0]:.2%})')
plt.ylabel(f'PC2 ({explained_variance[1]:.2%})')
plt.title('PCA Scatter Plot by Year')
plt.tight_layout()
plt.savefig('pca_scatter_year.png')
plt.close()


PCA Component Loadings:
                 PC1       PC2       PC3       PC4       PC5       PC6  \
AVG_L813    0.843645 -0.272144 -0.104019 -0.290623  0.178261 -0.008753   
AVG_M601    0.695737  0.594457  0.098555 -0.115289 -0.353037  0.046486   
AVG_SCI703  0.888575 -0.197225  0.079736  0.036675 -0.114406 -0.376253   
AVG_SST605  0.797860 -0.328597  0.005556  0.456936 -0.116388  0.147207   
AVG_M801    0.730700  0.453997  0.311312  0.151297  0.374149 -0.004490   
AVG_SCI801  0.839614 -0.289847  0.208448 -0.244045 -0.039555  0.208464   
AVG_SST704  0.709747  0.232554 -0.646129  0.031821  0.069842  0.028423   

                 PC7  
AVG_L813   -0.296235  
AVG_M601   -0.116018  
AVG_SCI703  0.099342  
AVG_SST605 -0.109984  
AVG_M801    0.028996  
AVG_SCI801  0.252384  
AVG_SST704  0.136699  


In [22]:
chi_square_value, p_value = calculate_bartlett_sphericity(df[performance_metrics])
kmo_all, kmo_model = calculate_kmo(df[performance_metrics])

print('\nBartlett test of sphericity:')
print(f'Chi-square value: {chi_square_value:.2f}')
print(f'p-value: {p_value:.6f}')
print('\nKMO Measure of Sampling Adequacy:', round(kmo_model, 3))

# Determine number of factors
fa = FactorAnalyzer(rotation=None)
fa.fit(df[performance_metrics])
ev, v = fa.get_eigenvalues()

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(ev) + 1), ev, 'bo-')
plt.axhline(y=1, color='r', linestyle='--')
plt.xlabel('Factor Number')
plt.ylabel('Eigenvalue')
plt.title('Scree Plot')
plt.grid(True)
plt.tight_layout()
plt.savefig('factor_analysis_scree_plot.png')
plt.close()


Bartlett test of sphericity:
Chi-square value: 6276.29
p-value: 0.000000

KMO Measure of Sampling Adequacy: 0.862


In [24]:
n_factors = sum(ev > 1)
print(f"\nOptimal number of factors (Kaiser criterion): {n_factors}")

fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
fa.fit(df[performance_metrics])

factor_df = pd.DataFrame(fa.loadings_, index=performance_metrics, columns=[f'Factor{i+1}' for i in range(n_factors)])
print("\nFactor Loadings:")
print(factor_df)

# Heatmap of factor loadings
plt.figure(figsize=(12, 8))
sns.heatmap(factor_df, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Factor Loadings')
plt.tight_layout()
plt.savefig('factor_loadings.png')
plt.close()


Optimal number of factors (Kaiser criterion): 1

Factor Loadings:
             Factor1
AVG_L813   -0.821579
AVG_M601   -0.624586
AVG_SCI703 -0.889933
AVG_SST605 -0.758835
AVG_M801   -0.665995
AVG_SCI801 -0.817035
AVG_SST704 -0.642995


In [26]:
df_top_states = df[df['State'].isin(top_states)]

# MANOVA: Year
formula = f"{'+'.join(performance_metrics)} ~ Year_Simple"
manova_year = MANOVA.from_formula(formula, data=df)
print("\nMANOVA - Effect of Year on Academic Performance:")
print(manova_year.mv_test())

# MANOVA: State
formula = f"{'+'.join(performance_metrics)} ~ State"
manova_state = MANOVA.from_formula(formula, data=df_top_states)
print("\nMANOVA - Effect of State on Academic Performance (Top 5 States):")
print(manova_state.mv_test())

# MANOVA: Year * State
formula = f"{'+'.join(performance_metrics)} ~ Year_Simple * State"
manova_interaction = MANOVA.from_formula(formula, data=df_top_states)
print("\nMANOVA - Interaction Effect of Year and State on Academic Performance:")
print(manova_interaction.mv_test())


MANOVA - Effect of Year on Academic Performance:
                    Multivariate linear model
                                                                 
-----------------------------------------------------------------
       Intercept         Value  Num DF   Den DF   F Value  Pr > F
-----------------------------------------------------------------
          Wilks' lambda  0.0374 7.0000 1442.0000 5303.0749 0.0000
         Pillai's trace  0.9626 7.0000 1442.0000 5303.0749 0.0000
 Hotelling-Lawley trace 25.7431 7.0000 1442.0000 5303.0749 0.0000
    Roy's greatest root 25.7431 7.0000 1442.0000 5303.0749 0.0000
-----------------------------------------------------------------
                                                                 
-----------------------------------------------------------------
        Year_Simple       Value  Num DF   Den DF  F Value  Pr > F
-----------------------------------------------------------------
            Wilks' lambda 0.3548 7.0000 1442.0

In [28]:
print("\nFollow-up ANOVAs for Year Effect:")
for metric in performance_metrics:
    model = ols(f'{metric} ~ Year_Simple', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(f"\nANOVA - Effect of Year on {metric}:")
    print(anova_table)

    # Means
    year_means = df.groupby('Year_Simple')[metric].mean()
    print(f"Mean {metric} by Year:")
    print(year_means)


Follow-up ANOVAs for Year Effect:

ANOVA - Effect of Year on AVG_L813:
                   sum_sq      df          F        PR(>F)
Year_Simple   2484.490750     1.0  38.369148  7.614215e-10
Residual     93761.337536  1448.0        NaN           NaN
Mean AVG_L813 by Year:
Year_Simple
2017    55.042450
2021    52.424278
Name: AVG_L813, dtype: float64

ANOVA - Effect of Year on AVG_M601:
                    sum_sq      df           F        PR(>F)
Year_Simple   38800.867268     1.0  431.584552  4.126927e-84
Residual     130179.950941  1448.0         NaN           NaN
Mean AVG_M601 by Year:
Year_Simple
2017    38.800065
2021    49.146725
Name: AVG_M601, dtype: float64

ANOVA - Effect of Year on AVG_SCI703:
                   sum_sq      df          F        PR(>F)
Year_Simple   4767.466244     1.0  80.356247  9.411851e-19
Residual     85908.580330  1448.0        NaN           NaN
Mean AVG_SCI703 by Year:
Year_Simple
2017    42.038295
2021    38.411498
Name: AVG_SCI703, dtype: float64

ANOV

In [30]:
# Create a function to test for normality and homogeneity of variance
def check_assumptions(df, dv, group_var):
    print(f"Assumption Testing for {dv} by {group_var}")
    
    # Test for normality in each group
    groups = df[group_var].unique()
    for group in groups:
        group_data = df[df[group_var] == group][dv]
        # Sample for Shapiro-Wilk test (maximum 5000 observations)
        if len(group_data) > 5000:
            group_data = group_data.sample(5000)
        # Further sample if still too large (maximum 100 for Shapiro-Wilk)
        if len(group_data) > 100:
            group_data = group_data.sample(100)
        
        stat, p = stats.shapiro(group_data)
        print(f"Shapiro-Wilk Test for {group}: Statistic={stat:.4f}, p-value={p:.4f}")
    
    # Test for homogeneity of variance
    # Gather data for each group
    group_data = [df[df[group_var] == group][dv] for group in groups]
    
    # Sample if any group is too large (max 5000)
    group_data = [data.sample(min(5000, len(data))) for data in group_data]
    
    levene_stat, levene_p = stats.levene(*group_data)
    print(f"Levene's Test: Statistic={levene_stat:.4f}, p-value={levene_p:.4f}")
    print()

# Check assumptions for a couple of examples
for metric in performance_metrics[:2]:  # Just check the first two for brevity
    check_assumptions(df, metric, 'Year_Simple')

# Visualize differences between years for each performance metric
plt.figure(figsize=(14, 10))
for i, metric in enumerate(performance_metrics, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x='Year_Simple', y=metric, data=df)
    plt.title(f'{metric} by Year')
plt.tight_layout()
plt.savefig('year_differences.png')
plt.close()

# Research Question: Has there been a significant improvement in academic performance from 2017 to 2021?
# Calculate the mean difference for each performance metric
year_diff = {}
for metric in performance_metrics:
    mean_2017 = df[df['Year_Simple'] == '2017'][metric].mean()
    mean_2021 = df[df['Year_Simple'] == '2021'][metric].mean()
    diff = mean_2021 - mean_2017
    perc_change = (diff / mean_2017) * 100
    year_diff[metric] = {'Mean 2017': mean_2017, 'Mean 2021': mean_2021, 
                        'Difference': diff, 'Percentage Change': perc_change}

year_diff_df = pd.DataFrame(year_diff).T
print("\nPerformance Metrics Comparison between 2017 and 2021:")
print(year_diff_df)

# Visualize performance changes
plt.figure(figsize=(12, 8))
year_diff_df['Percentage Change'].plot(kind='bar', color='skyblue')
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Percentage Change in Academic Performance (2017 to 2021)')
plt.ylabel('Percentage Change (%)')
plt.xlabel('Performance Metric')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('percentage_change.png')
plt.close()

# Additional analysis: State-wise performance change
# Calculate percentage change for top 5 states
state_year_diff = {}
for state in top_states:
    state_df = df[df['State'] == state]
    
    state_year_diff[state] = {}
    for metric in performance_metrics:
        mean_2017 = state_df[state_df['Year_Simple'] == '2017'][metric].mean()
        mean_2021 = state_df[state_df['Year_Simple'] == '2021'][metric].mean()
        perc_change = ((mean_2021 - mean_2017) / mean_2017) * 100
        state_year_diff[state][metric] = perc_change

state_year_diff_df = pd.DataFrame(state_year_diff)
print("\nPercentage Change by State (2017 to 2021):")
print(state_year_diff_df)

# Visualize state-wise percentage changes
plt.figure(figsize=(14, 10))
sns.heatmap(state_year_diff_df, annot=True, cmap='RdYlGn', center=0, fmt='.1f')
plt.title('Percentage Change in Academic Performance by State (2017 to 2021)')
plt.tight_layout()
plt.savefig('state_percentage_change.png')
plt.close()

# Create a multi-index DataFrame for comprehensive state-year analysis
state_year_means = df.groupby(['State', 'Year_Simple'])[performance_metrics].mean()
state_year_means = state_year_means.unstack(level='Year_Simple')
print("\nState-wise Performance Metrics by Year:")
print(state_year_means.head())

# Export results to CSV for later use
year_diff_df.to_csv('performance_change_results.csv')
state_year_diff_df.to_csv('state_performance_change_results.csv')

Assumption Testing for AVG_L813 by Year_Simple
Shapiro-Wilk Test for 2017: Statistic=0.9570, p-value=0.0025
Shapiro-Wilk Test for 2021: Statistic=0.9901, p-value=0.6742
Levene's Test: Statistic=0.1122, p-value=0.7377

Assumption Testing for AVG_M601 by Year_Simple
Shapiro-Wilk Test for 2017: Statistic=0.9655, p-value=0.0102
Shapiro-Wilk Test for 2021: Statistic=0.9901, p-value=0.6703
Levene's Test: Statistic=7.8301, p-value=0.0052


Performance Metrics Comparison between 2017 and 2021:
            Mean 2017  Mean 2021  Difference  Percentage Change
AVG_L813    55.042450  52.424278   -2.618172          -4.756641
AVG_M601    38.800065  49.146725   10.346660          26.666605
AVG_SCI703  42.038295  38.411498   -3.626797          -8.627365
AVG_SST605  47.885646  39.394597   -8.491049         -17.731929
AVG_M801    31.218705  33.149701    1.930997           6.185384
AVG_SCI801  50.030550  45.845367   -4.185183          -8.365256
AVG_SST704  39.017348  42.815399    3.798052           9.7342